In [1]:
import geopandas as gpd
import matplotlib.pyplot as plt
import matplotlib.patches as mpatches
import pandas as pd
import numpy as np
from shapely.ops import nearest_points
from shapely import union_all
from sklearn.neighbors import NearestNeighbors

In [2]:
url_eaton_buildings_study="https://raw.githubusercontent.com/heather-math/eaton/main/eaton_buildings_study.geojson"

In [3]:
gdf_eaton_buildings=gpd.read_file(url_eaton_buildings_study)

In [4]:
gdf_firezone=gpd.read_file("https://services.arcgis.com/RmCCgQtiZLDCtblq/arcgis/rest/services/Palisades_and_Eaton_Dissolved_Fire_Perimeters_as_of_20250121/FeatureServer/0/query?outFields=*&where=1%3D1&f=geojson")
gdf_firezone=gdf_firezone.to_crs(epsg=26911)
fire_geom = gdf_firezone.geometry.union_all()
gdf_eaton_buildings = gdf_eaton_buildings[gdf_eaton_buildings.intersects(fire_geom)]
print(gdf_eaton_buildings["DAMAGE"].value_counts())

DAMAGE
Destroyed (>50%)    8758
No Damage           2891
Affected (1-9%)      689
Minor (10-25%)       114
Major (26-50%)        64
Name: count, dtype: int64


## Add a column to gdf_eaton_buildings to record the average distance to the 3 nearest neighbors.

In [5]:
centroids = gdf_eaton_buildings.geometry.centroid
coords = np.column_stack([centroids.x.values, centroids.y.values])

In [6]:
nbrs = NearestNeighbors(n_neighbors=4, algorithm="ball_tree")
nbrs.fit(coords)
dist_centroid, idx_neighbors = nbrs.kneighbors(coords)

In [7]:
n = len(gdf_eaton_buildings)
nearest_wall = np.empty(n, dtype=float)
avg3_wall = np.empty(n, dtype=float)

geoms = gdf_eaton_buildings.geometry.values  # for speed

In [8]:
# For each building, compute tree wall-to-wall distance
for i in range(n):
    # First neighbor is the building itself (distance 0); skip it
    neighbor_idx = idx_neighbors[i, 1:]     # 3 nearest other buildings

    this_geom = geoms[i]
    # Compute polygon–polygon distances to the 3 neighbors
    dists = np.array([this_geom.distance(geoms[j]) for j in neighbor_idx])

    nearest_wall[i] = dists.min()
    avg3_wall[i] = dists.mean()

In [9]:
gdf_eaton_buildings["dist_1nn"] = nearest_wall          # nearest building wall-to-wall distance (meters)
gdf_eaton_buildings["dist_3nn_avg"] = avg3_wall         # mean of 3 nearest building distances (meters)

In [10]:
gdf_eaton_buildings

,OBJECTID,CODE,BLD_ID,HEIGHT,ELEV,SOURCE,DATE_,STATUS,OLD_BLD_ID,AREA,DAMAGE,STRUCTURET,Shape__Area,Shape__Length,geometry,dist_1nn,dist_3nn_avg
0,6244,Courtyard,None,NaN,NaN,LARIAC2,2008,Unchanged,None,41.282097,Destroyed (>50%),Single Family Residence Single Story,5.621094,9.731434,"POLYGON ((394984.482 3784728.835, 394986.746 3...",0.000000,5.823572
1,6245,Courtyard,None,NaN,NaN,LARIAC2,2008,Unchanged,None,62.494969,Destroyed (>50%),Single Family Residence Multi Story,8.511719,11.733945,"POLYGON ((394962.367 3785658.804, 394962.191 3...",0.000000,14.112906
2,6246,Courtyard,None,NaN,NaN,LARIAC2,2008,Unchanged,None,409.996462,Destroyed (>50%),Single Family Residence Multi Story,55.812500,29.883767,"POLYGON ((394957.071 3783933.159, 394962.787 3...",0.000000,17.520271
3,6247,Courtyard,None,NaN,NaN,LARIAC2,2008,Unchanged,None,221.369612,No Damage,Single Family Residence Single Story,30.132812,22.132594,"POLYGON ((396955.417 3783439.417, 396950.26 37...",0.000000,7.086712
4,6248,Courtyard,None,NaN,NaN,LARIAC2,2008,Unchanged,None,729.504391,No Damage,Multi Family Residence Multi Story,99.292969,59.170204,"POLYGON ((395841.867 3783670.528, 395838.729 3...",0.000000,8.901571
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
17258,45412,Building,202000124797,23.80,5456.07,LARIAC6,2020,Unchanged,201700192934,6254.918630,No Damage,Infrastructure,851.917969,128.775312,"POLYGON ((402168.516 3786305.951, 402165.608 3...",38.836727,979.278950
17261,45419,Building,202300214412,9.50,2896.84,LARIAC7,2023,Modified,535522894110,525.217781,Destroyed (>50%),Infrastructure,71.503906,35.738552,"POLYGON ((399890.921 3784544.109, 399900.617 3...",426.280185,454.990352
17262,45421,Building,202300214502,35.84,5741.61,LARIAC7,2023,Modified,541478904889,2402.307059,No Damage,Infrastructure,327.296875,75.306229,"POLYGON ((401734.182 3787802.528, 401744.316 3...",1.819819,24.431181
17263,45424,Building,202300214505,36.70,5726.96,LARIAC7,2023,Modified,541301904732,4338.158800,No Damage,Infrastructure,591.035156,110.564729,"POLYGON ((401683.428 3787768.772, 401709.789 3...",18.691476,40.689602


## Adding columns related to trees

In [11]:
tree_filepath="FOUR_INCH_TREES_IN_FIRE.gpkg"

In [12]:
gdf_trees = gpd.read_file(tree_filepath)

In [13]:
# Give each building an ID
gdf_eaton_buildings = gdf_eaton_buildings.reset_index(drop=True)
gdf_eaton_buildings["bldg_id"] = np.arange(len(gdf_eaton_buildings))

In [14]:
# --- Precompute buffers ---
buf_1_5 = gdf_eaton_buildings.geometry.buffer(1.5)
buf_3   = gdf_eaton_buildings.geometry.buffer(3.0)
buf_10  = gdf_eaton_buildings.geometry.buffer(10.0)

# Disjoint rings:
# 0–1.5 m 
ring_0_1_5 = buf_1_5.difference(gdf_eaton_buildings.geometry)

# B: 1.5–3 m
ring_1_5_3 = buf_3.difference(buf_1_5)

# C: 3–10 m 
ring_3_10  = buf_10.difference(buf_3)

In [15]:
# --- Build a single bands GeoDataFrame ---
r0_1_5 = gpd.GeoDataFrame(
    {"bldg_id": gdf_eaton_buildings["bldg_id"], "band": "0_1_5", "geometry": ring_0_1_5},
    crs=gdf_eaton_buildings.crs,
)
r1_5_3 = gpd.GeoDataFrame(
    {"bldg_id": gdf_eaton_buildings["bldg_id"], "band": "1_5_3", "geometry": ring_1_5_3},
    crs=gdf_eaton_buildings.crs,
)
r3_10 = gpd.GeoDataFrame(
    {"bldg_id": gdf_eaton_buildings["bldg_id"], "band": "3_10", "geometry": ring_3_10},
    crs=gdf_eaton_buildings.crs,
)
bands = gpd.pd.concat([r0_1_5, r1_5_3, r3_10], ignore_index=True)

In [16]:
# Precompute ring areas (denominator)
bands["ring_area"] = bands.geometry.area

In [17]:
ring_area_series = bands.set_index(["bldg_id", "band"])["ring_area"]

In [18]:
# --- overlay of all bands with tree cover ---
inter = gpd.overlay(
    bands[["bldg_id", "band", "geometry"]],
    gdf_trees[["geometry"]],
    how="intersection",
)

In [19]:
if inter.empty:
    # No trees intersect any ring: tree area is zero everywhere
    tree_area_series = ring_area_series * 0.0
    
else:
    inter["tree_area"] = inter.geometry.area
    tree_area_series = (
        inter.groupby(["bldg_id", "band"])["tree_area"]
        .sum()
    )
    # Make sure every (bldg_id, band) appears, missing → 0
    tree_area_series = tree_area_series.reindex(ring_area_series.index).fillna(0.0)

In [20]:
ring_area_wide = ring_area_series.unstack("band")   # area of each ring
tree_area_wide = tree_area_series.unstack("band")   # tree area in each ring

# Ensure all expected bands exist
for col in ["0_1_5", "1_5_3", "3_10"]:
    if col not in ring_area_wide.columns:
        ring_area_wide[col] = 0.0
    if col not in tree_area_wide.columns:
        tree_area_wide[col] = 0.0

# Build a single stats table with area_* and tree_area_* columns
stats = pd.concat(
    [
        ring_area_wide.add_prefix("area_"),       # area_0_1_5, area_1_5_3, area_3_10
        tree_area_wide.add_prefix("tree_area_"),  # tree_area_0_1_5, tree_area_1_5_3, tree_area_3_10
    ],
    axis=1,
)

In [21]:
# Join back to buildings on bldg_id
gdf_eaton_buildings = gdf_eaton_buildings.merge(
    stats,
    left_on="bldg_id",
    right_index=True,
    how="left",
)

In [22]:
# ---Compute percentages for disjoint bands ----
gdf_eaton_buildings["tree_pct_0_1_5m"] = 100.0 * gdf_eaton_buildings["tree_area_0_1_5"] / gdf_eaton_buildings["area_0_1_5"]
gdf_eaton_buildings["tree_pct_1_5_to_3m"] = 100.0 * gdf_eaton_buildings["tree_area_1_5_3"] / gdf_eaton_buildings["area_1_5_3"]
gdf_eaton_buildings["tree_pct_3_to_10m"] = 100.0 * gdf_eaton_buildings["tree_area_3_10"] / gdf_eaton_buildings["area_3_10"]

In [23]:
#  Compute percentages in combined regions
#     - within 1.5 m
#     - within 3 m (0–3 = combine 0–1.5 and 1.5–3)
#     - within 10 m (combine all three bands)
#     - within 1.5-10 m (combine 1.5-3 and 3-10)
# ---------------------------------------------------------------------
area_0_1_5 = gdf_eaton_buildings["area_0_1_5"]
area_1_5_3 = gdf_eaton_buildings["area_1_5_3"]
area_3_10  = gdf_eaton_buildings["area_3_10"]

tree_0_1_5 = gdf_eaton_buildings["tree_area_0_1_5"]
tree_1_5_3 = gdf_eaton_buildings["tree_area_1_5_3"]
tree_3_10  = gdf_eaton_buildings["tree_area_3_10"]


# within 1.5 m is just the 0–1.5 band
gdf_eaton_buildings["tree_pct_within_1_5m"] = gdf_eaton_buildings["tree_pct_0_1_5m"]

# within 3 m: combine 0–1.5 and 1.5–3
area_0_3 = area_0_1_5 + area_1_5_3
tree_0_3 = tree_0_1_5 + tree_1_5_3
gdf_eaton_buildings["tree_pct_within_3m"] = 100.0 * tree_0_3 / area_0_3

# within 10 m: combine all three bands
area_0_10 = area_0_1_5 + area_1_5_3 + area_3_10
tree_0_10 = tree_0_1_5 + tree_1_5_3 + tree_3_10
gdf_eaton_buildings["tree_pct_within_10m"] = 100.0 * tree_0_10 / area_0_10

# within 1.5-10m
area_1_5_10=area_1_5_3+area_3_10
tree_1_5_10=tree_1_5_3+tree_3_10
gdf_eaton_buildings["tree_pct_1_5_to_10m"] = 100.0 * tree_1_5_10 / area_1_5_10


# replace NaNs with 0:
for col in [
     "tree_pct_0_1_5m",
     "tree_pct_1_5_to_3m",
     "tree_pct_3_to_10m",
     "tree_pct_within_1_5m",
     "tree_pct_within_3m",
     "tree_pct_within_10m",
    "tree_pct_1_5_to_10m",
 ]:
    gdf_eaton_buildings[col] = gdf_eaton_buildings[col].fillna(0.0)


In [24]:
gdf_eaton_buildings.columns

Index(['OBJECTID', 'CODE', 'BLD_ID', 'HEIGHT', 'ELEV', 'SOURCE', 'DATE_',
       'STATUS', 'OLD_BLD_ID', 'AREA', 'DAMAGE', 'STRUCTURET', 'Shape__Area',
       'Shape__Length', 'geometry', 'dist_1nn', 'dist_3nn_avg', 'bldg_id',
       'area_0_1_5', 'area_1_5_3', 'area_3_10', 'tree_area_0_1_5',
       'tree_area_1_5_3', 'tree_area_3_10', 'tree_pct_0_1_5m',
       'tree_pct_1_5_to_3m', 'tree_pct_3_to_10m', 'tree_pct_within_1_5m',
       'tree_pct_within_3m', 'tree_pct_within_10m', 'tree_pct_1_5_to_10m'],
      dtype='object')

In [25]:
gdf_eaton_buildings.to_file("eaton_buildings_augmented.gpkg")